# CoT Explainability for RAG

Dieses Notebook zeigt, wie der Explainability-Prompt mit Retrieval-Trace genutzt wird.

basierend auf: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=11053836&tag=1


## Schritte
1. Sample-Retrieval-Trace definieren (hier: Dummy-Daten).
2. Messages mit `LLMClient.build_explainable_messages` bauen.
3. Optional: LLM aufrufen (Ollama/OpenAI) – deaktiviert, bis du `RUN_LLM=True` setzt.
4. Optional: Kleines RAG-Demo mit `RAGEngine` aktivieren, wenn Embeddings/Chroma lokal vorhanden sind.


In [1]:
import sys
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [2]:
from langchain_core.documents import Document
from src.modules.rag_engine import RAGEngine
from src.modules.llm_client import LLMClient
from src.modules.data_loader import DataLoader
from src.modules.data_loader_single_hop import BoolQDataLoader

### Small mock example

In [3]:
question = "Wer war die Ehepartnerin von Albert Einstein?"


client = LLMClient(provider="ollama", model_name="gemma3:1b")

demo_docs = [
    Document(page_content="Albert Einstein war ein theoretischer Physiker. Er heiratete Mileva Marić und später Elsa Einstein.", metadata={"title": "Albert Einstein"}),
    Document(page_content="Mileva Marić war eine serbische Physikerin und die erste Ehefrau von Albert Einstein.", metadata={"title": "Mileva Marić"}),
]

engine = RAGEngine(persist_dir="data/vector_db_demo")
engine.setup(documents=demo_docs, reset=True)

live_trace = engine.retrieve_with_scores(question, k=3)
messages = client.build_explainable_messages(question, live_trace)
llm = client.get_llm()
response = llm.invoke(messages)
print(response)



Creating new vector store with 2 documents...
Split into 2 chunks.
RagEngine ready.
🔌 Connecting to local Ollama (gemma3:1b)...
content='Antwort: Mileva Marić\nBegründung: Der Kontextchunk [chunk-2] erwähnt, dass Mileva Marić Albert Eists Ehefrau war.' additional_kwargs={} response_metadata={'model': 'gemma3:1b', 'created_at': '2025-12-15T15:12:24.246968Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3441669042, 'load_duration': 1378671667, 'prompt_eval_count': 260, 'prompt_eval_duration': 654218959, 'eval_count': 35, 'eval_duration': 1305153789, 'logprobs': None, 'model_name': 'gemma3:1b', 'model_provider': 'ollama'} id='lc_run--019b2291-d341-7a71-ae2e-8a6955eb5ac4-0' usage_metadata={'input_tokens': 260, 'output_tokens': 35, 'total_tokens': 295}


### Real data example

In [4]:
_dataLoader = BoolQDataLoader()
documents = _dataLoader.setup()

RuntimeError: Failed to download BoolQ split 'train': 403 Client Error: Forbidden for url: https://storage.googleapis.com/boolq/train.jsonl

In [ ]:
ragEngine = RAGEngine()
ragEngine.setup(documents=documents)

Loading existing vector store from ../data/vector_db...
RagEngine ready.


In [ ]:
client = LLMClient(provider="ollama", model_name="gemma3:1b")

question = "do good samaritan laws protect those who help at an accident?"

live_trace = ragEngine.retrieve_with_scores(question)

print(live_trace[0])
print(live_trace[1])
print(live_trace[2])
print(live_trace[3])

messages = client.build_explainable_messages(question, live_trace)
llm = client.get_llm()
response = llm.invoke(messages)
print(response)

{'id': 'chunk-1', 'score': 0.7372868921140145, 'title': 'ShowBiz Pizza Place', 'content': "ShowBiz Pizza Place was a restaurant pizza chain and family entertainment center founded in 1980 by Robert L. Brock and Creative Engineering.  The brand emerged following a separation between Brock and Pizza Time Theatre, owners of the Chuck E. Cheese's franchise.  ShowBiz Pizza restaurants entertained guests through a large selection of arcade games, coin-operated rides, and animatronic stage shows as a way to provide a complete package of food and entertainment.", 'metadata': {'is_supporting': False, 'title': 'ShowBiz Pizza Place', 'question_id': '5ab80e565542991d32223821', 'gold_sentence_indices': '', 'question': "Which franchise was founded in 1978, Chuck E. Cheese's or Jet's Pizza?", 'answer': '', 'source': 'hotpotqa'}}
{'id': 'chunk-2', 'score': 0.5226932626414018, 'title': "Chuck E. Cheese's", 'content': "Chuck E. Cheese's (officially Chuck E. Cheese Pizzeria & Games) is a chain of America